# Class Metaprogramming
Class metaprogramming is the art of creating or customizing classes at runtime. Classes are first-class objects in Python, so a function can be used to create a new class at any time, without using the class keyword.

Metaclasses are powerful, but hard to get right. Class decorators solve many of the same
problems more simply.

## A Class Factory
The standard library has a class factory that we’ve seen several times in this book:
collections.namedtuple.

In [3]:
def record_factory(cls_name, field_names):
    try:
        field_names = field_names.replace(',', ' ').split()
    except AttributeError: # no .replace or .split
        pass # assume it's already a sequence of identifiers
    field_names = tuple(field_names)

    def __init__(self, *args, **kwargs):
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)
 
    def __iter__(self):
        for name in self.__slots__:
            yield getattr(self, name)

    def __repr__(self):
        values = ', '.join('{}={!r}'.format(*i) for i in zip(self.__slots__, self))
        return '{}({})'.format(self.__class__.__name__, values)

    cls_attrs = dict(__slots__ = field_names,
                     __init__ = __init__,
                     __iter__ = __iter__,
                     __repr__ = __repr__)
    return type(cls_name, (object,), cls_attrs)


In [5]:
Dog = record_factory('Dog', 'name weight owner')

rex = Dog('Rex', 30, 'Bob')

In [6]:
rex

Dog(name='Rex', weight=30, owner='Bob')

type is a class. It behaves like a class that creates a new class when invoked with three arguments...

In [14]:
MySuperClass = object

MyClass = type('MyClass', (MySuperClass,), {'x': 42, 'x2': lambda self: self.x * 2})

In [16]:
MyClass.__dict__

mappingproxy({'x': 42,
              'x2': <function __main__.<lambda>(self)>,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

## A Class Decorator for Customizing Descriptors